In [ ]:
# prompt: set up spark -3.4.3

!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz
!tar xf spark-3.4.3-bin-hadoop3.tgz
!pip install -q findspark
!export PATH=$PATH:/content/spark-3.4.3-bin-hadoop3/bin
!export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [ ]:
# Set Spark Home
os.environ["SPARK_HOME"] = "/content/spark-3.4.3-bin-hadoop3"

# Create a SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("analysis").getOrCreate()

# Print the SparkSession
print(spark)

In [ ]:
spark

In [ ]:
posting=spark.read.format("csv").option("header","true").load("/content/marketing_sample_for_naukri_com-jobs__20190701_20190830__30k_data.csv")

In [ ]:
posting.show()

+--------------------+--------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             Uniq Id|     Crawl Timestamp|           Job Title|          Job Salary|Job Experience Required|          Key Skills|       Role Category|            Location|     Functional Area|            Industry|                Role|
+--------------------+--------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|9be62c49a0b7ebe98...|2019-07-05 01:46:...| Digital Media Pl...| Not Disclosed by...|             5 - 10 yrs| Media Planning| ...|         Advertising|              Mumbai|Marketing , Adver...|Advertising, PR, ...|Media Planning Ex...|
|3c52d436e39f596b2...|2019-07-06 08:04:...| Online Biddi

In [ ]:
posting.head()

Row(Uniq Id='9be62c49a0b7ebe982a4af1edaa7bc5f', Crawl Timestamp='2019-07-05 01:46:07 +0000', Job Title=' Digital Media Planner', Job Salary=' Not Disclosed by Recruiter ', Job Experience Required='5 - 10 yrs', Key Skills=' Media Planning| Digital Media', Role Category='Advertising', Location='Mumbai', Functional Area='Marketing , Advertising , MR , PR , Media Planning', Industry='Advertising, PR, MR, Event Management', Role='Media Planning Executive/Manager')

In [ ]:
from pyspark.sql.functions import col, sum

posting.select([sum(col(c).isNull().cast("int")).alias(c) for c in posting.columns]).show()

+-------+---------------+---------+----------+-----------------------+----------+-------------+--------+---------------+--------+----+
|Uniq Id|Crawl Timestamp|Job Title|Job Salary|Job Experience Required|Key Skills|Role Category|Location|Functional Area|Industry|Role|
+-------+---------------+---------+----------+-----------------------+----------+-------------+--------+---------------+--------+----+
|      0|              0|      575|        50|                    573|      1271|         2304|     577|            573|     573| 900|
+-------+---------------+---------+----------+-----------------------+----------+-------------+--------+---------------+--------+----+



In [ ]:
posting = posting.dropna()

In [ ]:
from pyspark.sql.functions import col

posting = posting.filter((col("Uniq Id") != "") &
                         ((col("Job Title") != "") |
                          (col("Job Salary") != "") |
                          (col("Job Experience Required") != "") |
                          (col("Key Skills") != "") |
                          (col("Role Category") != "") |
                          (col("Location") != "") |
                          (col("Functional Area") != "") |
                          (col("Industry") != "") |
                          (col("Role") != "")))

In [ ]:
posting.printSchema()

root
 |-- Uniq Id: string (nullable = true)
 |-- Crawl Timestamp: string (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Job Salary: string (nullable = true)
 |-- Job Experience Required: string (nullable = true)
 |-- Key Skills: string (nullable = true)
 |-- Role Category: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Functional Area: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Role: string (nullable = true)



In [ ]:
unique_job_titles = posting.select("Job Title").distinct()
unique_job_titles.show()

+--------------------+
|           Job Title|
+--------------------+
| Inbound Technica...|
| Adeeba Group is ...|
|   Embedded Engineer|
| Group Leader - M...|
| Fresher Engineer...|
| Nutrition Execut...|
| J2EE Full Stack ...|
| System Administr...|
| DIS Technology A...|
|Assistant Manager...|
|Amazon Hiring Dat...|
| Opening for Seni...|
| Senior Executive...|
| HR Manager Jobs ...|
| Share Point Deve...|
|     Java Developer,|
| Pediatrician Con...|
| Team Leader : Cr...|
| Required 10th/12...|
|Supply Chain and ...|
+--------------------+
only showing top 20 rows



In [ ]:
unique_job_titles = posting.select("Job Salary").distinct()
unique_job_titles.show()

+--------------------+
|          Job Salary|
+--------------------+
| 1,25,000 - 4,25,...|
| 12,00,000 - 20,0...|
| INR 2,00,000 - 2...|
| INR Both Side Ca...|
| 2,75,000 - 5,50,...|
| 8,50,000 - 9,00,...|
| 6,00,000 - 13,00...|
| 18,00,000 - 20,0...|
| 9,00,000 - 15,00...|
|INR 5,00,000 - 10...|
| 7,50,000 - 10,00...|
| INR 2,50,000 - 4...|
| 1,00,000 - 2,25,...|
| INR 5,50,000 - 6...|
| 8,00,000 - 16,00...|
| At par with indu...|
| 1,50,000 - 4,00,...|
| Best in the Indu...|
| Best in the Market |
| 2,25,000 - 2,50,...|
+--------------------+
only showing top 20 rows



In [ ]:
unique_job_titles = posting.select("Job Experience Required").distinct()
unique_job_titles.show()

+-----------------------+
|Job Experience Required|
+-----------------------+
|              0 - 3 yrs|
|            1 - 3 Years|
|          14 - 16 Years|
|             2 - 10 yrs|
|          12 - 18 Years|
|              0 - 4 yrs|
|             7 - 12 yrs|
|              5 - 7 yrs|
|             9 - 12 yrs|
|            2 - 4 Years|
|            11 - 18 yrs|
|              2 - 7 yrs|
|                0 Years|
|            15 - 23 yrs|
|          14 - 22 Years|
|          12 - 20 Years|
|            17 - 27 yrs|
|              1 - 7 yrs|
|              1 - 4 yrs|
|           7 - 12 Years|
+-----------------------+
only showing top 20 rows



In [ ]:
unique_job_titles = posting.select("Key Skills").distinct()
unique_job_titles.show()

+--------------------+
|          Key Skills|
+--------------------+
| Outbound| Medica...|
|    land acquisition|
| Relationship man...|
| Outbound| Techni...|
| Angularjs| Java|...|
| Email| PDF| EMEA...|
| Application Soft...|
| Estimation| Tend...|
| QA automation| L...|
| Contract managem...|
| SAP SD| SAP SD C...|
| b2b sales| busin...|
| rfp| presales| r...|
| International Sa...|
| Web Technologies...|
| Attendance| Trai...|
| bd| Sales| Direc...|
| Secondary resear...|
| PHP| MySQL| SEO|...|
| Field Sales| Mar...|
+--------------------+
only showing top 20 rows



In [ ]:
unique_job_titles = posting.select("Role Category").distinct()
unique_job_titles.show()

+--------------------+
|       Role Category|
+--------------------+
| After Sales Service|
|  Investment Banking|
|"Xoriant Corporat...|
|Magma Fincorp Lim...|
|             Telecom|
|Operations/Processes|
|Allegis Group Ind...|
|At TEKsystems, we...|
|       Card Products|
|Company Profile H...|
|Programming & Design|
|               Voice|
|"* A US based $4....|
|Interaction with ...|
|Saisun Outsourcin...|
| Working with Org...|
|            Teachers|
|Mutual Funds/Fund...|
|Language Teachers...|
|           OYO ROOMS|
+--------------------+
only showing top 20 rows



In [ ]:
unique_job_titles = posting.select("Location").distinct()
unique_job_titles.show()

+--------------------+
|            Location|
+--------------------+
|Bengaluru,Chennai...|
|Bengaluru,Hyderab...|
| Bengaluru, Pune,...|
|Chennai,Delhi NCR...|
|Bhiwadi,Gurgaon,M...|
|Hyderabad,Pune,Ko...|
|Delhi NCR,Mumbai,...|
|Ahmedabad,Other C...|
|           Bangalore|
|    Vellore, Chennai|
|Hyderabad (Pochar...|
|Delhi NCR,Bengalu...|
|         Pune,Mumbai|
|Mumbai,Bengaluru,...|
|Bengaluru,Delhi,J...|
|Bengaluru,Ahmedab...|
|Mumbai (Prabhadev...|
|Ahmedabad,Surat,J...|
|Faridabad,Delhi,M...|
|Delhi NCR,Bengalu...|
+--------------------+
only showing top 20 rows



In [ ]:
unique_job_titles = posting.select("Functional Area").distinct()
unique_job_titles.show()

+--------------------+
|     Functional Area|
+--------------------+
|Architecture , In...|
|IT Software - Sys...|
|IT Software - Mid...|
|TV , Films , Prod...|
|Accounts , Financ...|
|Allegis Group Ind...|
|Export , Import ,...|
|At TEKsystems, we...|
|Teaching , Educat...|
|Production , Manu...|
|Marketing , Adver...|
|IT Software - App...|
| Infrastructure a...|
| the company spec...|
|Sales , Retail , ...|
|Purchase / Logist...|
|000 colleagues in...|
| Infrastructure a...|
| share knowledge ...|
|Strategy , Manage...|
+--------------------+
only showing top 20 rows



In [ ]:
unique_job_titles = posting.select("Industry").distinct()
unique_job_titles.show()

+--------------------+
|            Industry|
+--------------------+
|Automobile / Auto...|
|  Retail / Wholesale|
|000 employees acr...|
|Allegis Group Ind...|
|NGO, Social Servi...|
|        Broadcasting|
| Government, Defence|
|Travel / Hotels /...|
|At TEKsystems, we...|
|Architecture / In...|
|IT-Software, Soft...|
|Recruitment, Staf...|
|Aviation, Aerospa...|
|Travel , Hotels ,...|
|Semiconductors, E...|
|Banking, Financia...|
| Brewery, Distillery|
|Consumer Electron...|
|Real Estate, Prop...|
|Media, Entertainm...|
+--------------------+
only showing top 20 rows



In [ ]:
unique_job_titles = posting.select("Role").distinct()
unique_job_titles.show()

+--------------------+
|                Role|
+--------------------+
|Assistant Manager...|
|        Ground Staff|
|    Product Designer|
|Head / VP / GM-Ta...|
|        Cardiologist|
|         RF Engineer|
|   Financial Analyst|
|Team Lead/Technic...|
|Tour Management M...|
|Database Architec...|
|Allegis Group Ind...|
|           Webmaster|
|  Sales / BD Manager|
|Structural Engine...|
|    Treasury Manager|
|Mechanical Engine...|
|At TEKsystems, we...|
|   Footwear Designer|
|Service Delivery ...|
|        2nd Engineer|
+--------------------+
only showing top 20 rows



In [ ]:
posting = posting.drop("Crawl Timestamp")

In [ ]:
posting.printSchema()

root
 |-- Uniq Id: string (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Job Salary: string (nullable = true)
 |-- Job Experience Required: string (nullable = true)
 |-- Key Skills: string (nullable = true)
 |-- Role Category: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Functional Area: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Role: string (nullable = true)



In [ ]:
from pyspark.sql.functions import split, explode
posting = posting.withColumn("Job Title", explode(split(posting["Job Title"], "/")))

# Show the DataFrame with split "Job Title"
posting.show()

+--------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             Uniq Id|           Job Title|          Job Salary|Job Experience Required|          Key Skills|       Role Category|            Location|     Functional Area|            Industry|                Role|
+--------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|9be62c49a0b7ebe98...| Digital Media Pl...| Not Disclosed by...|             5 - 10 yrs| Media Planning| ...|         Advertising|              Mumbai|Marketing , Adver...|Advertising, PR, ...|Media Planning Ex...|
|3c52d436e39f596b2...| Online Bidding E...| Not Disclosed by...|              2 - 5 yrs| pre sales| closi...|        Retail Sales|          

In [ ]:
split_salary = split(posting["Job Salary"], " - ")

# Extract highest and lowest salaries
df = posting.withColumn("Highest Salary", split_salary.getItem(1))
df = df.withColumn("Lowest Salary", split_salary.getItem(0))

# Show the DataFrame with extracted salaries
df.show(truncate=False)

+--------------------------------+--------------------------------------------------------------+----------------------------+-----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------+-----------------------------------+------------------------------------------------------+----------------------------------------------+---------------------------------------+--------------+----------------------------+
|Uniq Id                         |Job Title                                                     |Job Salary                  |Job Experience Required|Key Skills                                                                                                                                                              |Role Category                             |Location                           |Functional Area  

In [ ]:
# Print Job Salary
df.select("Job Salary").show(truncate=False)

# Print Highest Salary
df.select("Highest Salary").show(truncate=False)

# Print Lowest Salary
df.select("Lowest Salary").show(truncate=False)

df=df.drop("Highest Salary","Lowest Salary")

+----------------------------+
|Job Salary                  |
+----------------------------+
| Not Disclosed by Recruiter |
| Not Disclosed by Recruiter |
| Not Disclosed by Recruiter |
| Not Disclosed by Recruiter |
| 2,00,000 - 4,00,000 PA.    |
| Not Disclosed by Recruiter |
| 7,00,000 - 17,00,000 PA.   |
| 2,00,000 - 3,00,000 PA.    |
| Not Disclosed by Recruiter |
| Not Disclosed by Recruiter |
| Not Disclosed by Recruiter |
| Not Disclosed by Recruiter |
| 25,00,000 - 35,00,000 PA.  |
| Not Disclosed by Recruiter |
| Not Disclosed by Recruiter |
|Openings: 1                 |
| 3,50,000 - 5,00,000 PA.    |
| 3,50,000 - 5,00,000 PA.    |
| Not Disclosed by Recruiter |
| Not Disclosed by Recruiter |
+----------------------------+
only showing top 20 rows

+--------------+
|Highest Salary|
+--------------+
|null          |
|null          |
|null          |
|null          |
|4,00,000 PA.  |
|null          |
|17,00,000 PA. |
|3,00,000 PA.  |
|null          |
|null          |
|null    

In [ ]:

from pyspark.sql.functions import col, regexp_extract

pattern = r'(\d{1,2},\d{2},000) - (\d{1,2},\d{2},000) PA.'

# Extract highest and lowest salary using regular expression
extracted_salaries = df.select(
    col("Uniq Id"),
    regexp_extract(col("Job Salary"), pattern, 1).alias("Highest Salary"),
    regexp_extract(col("Job Salary"), pattern, 2).alias("Lowest Salary")
)

# Show the extracted salaries
extracted_salaries.show(truncate=False)


+--------------------------------+--------------+-------------+
|Uniq Id                         |Highest Salary|Lowest Salary|
+--------------------------------+--------------+-------------+
|9be62c49a0b7ebe982a4af1edaa7bc5f|              |             |
|3c52d436e39f596b22519da2612f6a56|              |             |
|ffad8a2396c60be2bf6d0e2ff47c58d4|              |             |
|ffad8a2396c60be2bf6d0e2ff47c58d4|              |             |
|7b921f51b5c2fb862b4a5f7a54c37f75|2,00,000      |4,00,000     |
|2d8b7d44e138a54d5dc841163138de50|              |             |
|c9628ea8d9cfd2ca72e14d53783aca90|7,00,000      |17,00,000    |
|2008eb365fa0ebcb1ccd9a50efc3da49|2,00,000      |3,00,000     |
|9642bed00e26183e41ab73778bf48108|              |             |
|2633768efe8c6f72d75158f1c173f893|              |             |
|138a9ab97a766d8048f5b246172f65ed|              |             |
|138a9ab97a766d8048f5b246172f65ed|              |             |
|823c36988178cf5c9a6a34e4fc75d4e0|25,00,

In [ ]:
# prompt: count row of extracted_salaries and df

print("Count of rows in extracted_salaries:", extracted_salaries.count())
print("Count of rows in df:", df.count())


Count of rows in extracted_salaries: 32646
Count of rows in df: 32646


In [ ]:
# Assuming df is your existing DataFrame and extracted_salaries contains the relevant data

# Joining df and extracted_salaries based on the common column "Uniq Id"
joined_df = df.join(extracted_salaries, on="Uniq Id", how="left")

# Showing the joined DataFrame
joined_df.show(truncate=False)


+--------------------------------+--------------------------------------------------------------+----------------------------+-----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------+-----------------------------------+------------------------------------------------------+----------------------------------------------+---------------------------------------+--------------+-------------+
|Uniq Id                         |Job Title                                                     |Job Salary                  |Job Experience Required|Key Skills                                                                                                                                                              |Role Category                             |Location                           |Functional Area                 

In [ ]:
from pyspark.sql.functions import col, regexp_extract

pattern = r'(\d+) - (\d+) yrs'

# Extract minimum and maximum experience required using regular expression
extracted_experience = df.select(
    col("Uniq Id"),
    regexp_extract(col("Job Experience Required"), pattern, 1).alias("Min Experience Required"),
    regexp_extract(col("Job Experience Required"), pattern, 2).alias("Max Experience Required")
)

# Show the extracted experience required
extracted_experience.show(truncate=False)


+--------------------------------+-----------------------+-----------------------+
|Uniq Id                         |Min Experience Required|Max Experience Required|
+--------------------------------+-----------------------+-----------------------+
|9be62c49a0b7ebe982a4af1edaa7bc5f|5                      |10                     |
|3c52d436e39f596b22519da2612f6a56|2                      |5                      |
|ffad8a2396c60be2bf6d0e2ff47c58d4|0                      |1                      |
|ffad8a2396c60be2bf6d0e2ff47c58d4|0                      |1                      |
|7b921f51b5c2fb862b4a5f7a54c37f75|0                      |5                      |
|2d8b7d44e138a54d5dc841163138de50|2                      |5                      |
|c9628ea8d9cfd2ca72e14d53783aca90|5                      |7                      |
|2008eb365fa0ebcb1ccd9a50efc3da49|0                      |0                      |
|9642bed00e26183e41ab73778bf48108|9                      |14                     |
|263

In [ ]:
df = joined_df.join(extracted_experience, on="Uniq Id", how="left")

# Showing the joined DataFrame
df.show(truncate=False)

+--------------------------------+--------------------------------------------------------------+----------------------------+-----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------+---------+------------------------------------------------------+-------------------------------------+---------------------------------------+--------------+-------------+-----------------------+-----------------------+
|Uniq Id                         |Job Title                                                     |Job Salary                  |Job Experience Required|Key Skills                                                                                                                                                              |Role Category                             |Location |Functional Area                              

In [ ]:
df.printSchema()

root
 |-- Uniq Id: string (nullable = true)
 |-- Job Title: string (nullable = false)
 |-- Job Salary: string (nullable = true)
 |-- Job Experience Required: string (nullable = true)
 |-- Key Skills: string (nullable = true)
 |-- Role Category: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Functional Area: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Role: string (nullable = true)
 |-- Highest Salary: string (nullable = true)
 |-- Lowest Salary: string (nullable = true)
 |-- Min Experience Required: string (nullable = true)
 |-- Max Experience Required: string (nullable = true)



In [ ]:
df.select("Key Skills").show()

+--------------------+
|          Key Skills|
+--------------------+
| Media Planning| ...|
| pre sales| closi...|
| Computer science...|
| Computer science...|
| Computer science...|
| Computer science...|
| Computer science...|
| Computer science...|
| Computer science...|
| Computer science...|
|   Technical Support|
| manual testing| ...|
| adobe experience...|
| channel partners...|
| TFS| Azure| Git|...|
|                 Bde|
| technical suppor...|
| technical suppor...|
| technical suppor...|
| technical suppor...|
+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import split, explode
df = df.withColumn("Location", explode(split(posting["Location"], ",")))

# Show the DataFrame with split "Job Title"
df.show()

+--------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------+-------------+-----------------------+-----------------------+
|             Uniq Id|           Job Title|          Job Salary|Job Experience Required|          Key Skills|       Role Category| Location|     Functional Area|            Industry|                Role|Highest Salary|Lowest Salary|Min Experience Required|Max Experience Required|
+--------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------+-------------+-----------------------+-----------------------+
|9be62c49a0b7ebe98...| Digital Media Pl...| Not Disclosed by...|             5 - 10 yrs| Media Planning| ...|         Advertising|   Mumbai|Marketing , Adver

In [ ]:
from pyspark.sql.functions import col, when

# Convert empty spaces in "Highest Salary" column to "Not disclosed"
df = df.withColumn("Highest Salary", when(col("Highest Salary") == "", "Not disclosed").otherwise(col("Highest Salary")))


In [ ]:
df = df.withColumn("Lowest Salary", when(col("Lowest Salary") == "", "Not disclosed").otherwise(col("Lowest Salary")))

In [ ]:
df.show()

+--------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------+-------------+-----------------------+-----------------------+
|             Uniq Id|           Job Title|          Job Salary|Job Experience Required|          Key Skills|       Role Category| Location|     Functional Area|            Industry|                Role|Highest Salary|Lowest Salary|Min Experience Required|Max Experience Required|
+--------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------+-------------+-----------------------+-----------------------+
|9be62c49a0b7ebe98...| Digital Media Pl...| Not Disclosed by...|             5 - 10 yrs| Media Planning| ...|         Advertising|   Mumbai|Marketing , Adver

In [ ]:
df = df.filter(
    ((col("Job Title") != "") |
     (col("Job Salary") != "") |
     (col("Job Experience Required") != "") |
     (col("Key Skills") != "") |
     (col("Role Category") != "") |
     (col("Location") != "") |
     (col("Functional Area") != "") |
     (col("Industry") != "") |
     (col("Role") != "") |
     (col("Highest Salary") != "") |
     (col("Lowest Salary") != "") |
     (col("Min Experience Required") != "") |
     (col("Max Experience Required") != ""))
)

# Show the filtered DataFrame
df.show()

+--------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------+-------------+-----------------------+-----------------------+
|             Uniq Id|           Job Title|          Job Salary|Job Experience Required|          Key Skills|       Role Category| Location|     Functional Area|            Industry|                Role|Highest Salary|Lowest Salary|Min Experience Required|Max Experience Required|
+--------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------+-------------+-----------------------+-----------------------+
|9be62c49a0b7ebe98...| Digital Media Pl...| Not Disclosed by...|             5 - 10 yrs| Media Planning| ...|         Advertising|   Mumbai|Marketing , Adver

In [ ]:

from pyspark.sql.functions import split, explode
df = df.withColumn("Functional Area", explode(split(posting["Functional Area"], ",")))

# Show the DataFrame with split "Job Title"
df.show()

+--------------------+--------------------+--------------------+-----------------------+--------------------+-------------+--------+--------------------+--------------------+--------------------+--------------+-------------+-----------------------+-----------------------+
|             Uniq Id|           Job Title|          Job Salary|Job Experience Required|          Key Skills|Role Category|Location|     Functional Area|            Industry|                Role|Highest Salary|Lowest Salary|Min Experience Required|Max Experience Required|
+--------------------+--------------------+--------------------+-----------------------+--------------------+-------------+--------+--------------------+--------------------+--------------------+--------------+-------------+-----------------------+-----------------------+
|9be62c49a0b7ebe98...| Digital Media Pl...| Not Disclosed by...|             5 - 10 yrs| Media Planning| ...|  Advertising|  Mumbai|          Marketing |Advertising, PR, ...|Media P

In [ ]:
from pyspark.sql.functions import split, explode
df = df.withColumn("Industry", explode(split(posting["Industry"], ",")))

# Show the DataFrame with split "Job Title"
df.show()

+--------------------+--------------------+--------------------+-----------------------+--------------------+-------------+--------+---------------+-----------------+--------------------+--------------+-------------+-----------------------+-----------------------+
|             Uniq Id|           Job Title|          Job Salary|Job Experience Required|          Key Skills|Role Category|Location|Functional Area|         Industry|                Role|Highest Salary|Lowest Salary|Min Experience Required|Max Experience Required|
+--------------------+--------------------+--------------------+-----------------------+--------------------+-------------+--------+---------------+-----------------+--------------------+--------------+-------------+-----------------------+-----------------------+
|9be62c49a0b7ebe98...| Digital Media Pl...| Not Disclosed by...|             5 - 10 yrs| Media Planning| ...|  Advertising|  Mumbai|     Marketing |      Advertising|Media Planning Ex...| Not disclosed|Not

In [ ]:
from pyspark.sql.functions import col


df = df.filter(
    ((col("Job Title") != "") |
     (col("Job Salary") != "") |
     (col("Job Experience Required") != "") |
     (col("Key Skills") != "") |
     (col("Role Category") != "") |
     (col("Location") != "") |
     (col("Functional Area") != "") |
     (col("Industry") != "") |
     (col("Role") != "") |
     (col("Highest Salary") != "") |
     (col("Lowest Salary") != "") |
     (col("Min Experience Required") != "") |
     (col("Max Experience Required") != ""))
)

# Show the filtered DataFrame
df.show()

+--------------------+--------------------+--------------------+-----------------------+--------------------+-------------+--------+---------------+-----------------+--------------------+--------------+-------------+-----------------------+-----------------------+
|             Uniq Id|           Job Title|          Job Salary|Job Experience Required|          Key Skills|Role Category|Location|Functional Area|         Industry|                Role|Highest Salary|Lowest Salary|Min Experience Required|Max Experience Required|
+--------------------+--------------------+--------------------+-----------------------+--------------------+-------------+--------+---------------+-----------------+--------------------+--------------+-------------+-----------------------+-----------------------+
|9be62c49a0b7ebe98...| Digital Media Pl...| Not Disclosed by...|             5 - 10 yrs| Media Planning| ...|  Advertising|  Mumbai|     Marketing |      Advertising|Media Planning Ex...| Not disclosed|Not

In [ ]:
# prompt: download df as csv
# Write DataFrame to a CSV file
df.write.csv("/content/job_data.csv", header=True, mode="overwrite")


In [ ]:
from pyspark.sql.functions import split, posexplode

# Split the "Key Skills" column into an array based on the delimiter "|"
df = df.withColumn("Key Skills", split(df["Key Skills"], "\\|"))

# Explode the array elements while maintaining their position
df = df.selectExpr("*", "posexplode(`Key Skills`) as (pos, skill)")

# Filter out odd indices to keep only the skills and remove the indices
df = df.filter(df["pos"] % 2 != 0).drop("pos")

# Show the DataFrame with split "Key Skills"
df.show(truncate=False)


AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "split(Key Skills, \|, -1)" due to data type mismatch: Parameter 1 requires the "STRING" type, however "Key Skills" has the type "ARRAY<STRING>".;
'Project [Uniq Id#7791, Job Title#8189, Job Salary#7794, Job Experience Required#7795, split(Key Skills#9454, \|, -1) AS Key Skills#9472, Role Category#7797, Location#8741, Functional Area#9027, Industry#9133, Role#7801, Highest Salary#9238, Lowest Salary#9253, Min Experience Required#8567, Max Experience Required#8568]
+- Project [Uniq Id#7791, Job Title#8189, Job Salary#7794, Job Experience Required#7795, split(Key Skills#7796, \|, -1) AS Key Skills#9454, Role Category#7797, Location#8741, Functional Area#9027, Industry#9133, Role#7801, Highest Salary#9238, Lowest Salary#9253, Min Experience Required#8567, Max Experience Required#8568]
   +- Filter ((((((((((((NOT (Job Title#8189 = ) OR NOT (Job Salary#7794 = )) OR NOT (Job Experience Required#7795 = )) OR NOT (Key

In [ ]:
# Explode the array elements while maintaining their position
df = df.selectExpr("*", "posexplode(`Key Skills`) as (pos, skill)")

# Filter out odd indices to keep only the skills and remove the indices
df = df.filter(df["pos"] % 2 != 0).drop("pos")

# Show the DataFrame with split "Key Skills"
df.show(truncate=False)

+--------------------------------+----------------------+----------------------------+-----------------------+---------------------------------+-------------+--------+---------------+-----------------+--------------------------------+--------------+-------------+-----------------------+-----------------------+--------------+
|Uniq Id                         |Job Title             |Job Salary                  |Job Experience Required|Key Skills                       |Role Category|Location|Functional Area|Industry         |Role                            |Highest Salary|Lowest Salary|Min Experience Required|Max Experience Required|skill         |
+--------------------------------+----------------------+----------------------------+-----------------------+---------------------------------+-------------+--------+---------------+-----------------+--------------------------------+--------------+-------------+-----------------------+-----------------------+--------------+
|9be62c49a0b7ebe982

In [ ]:
df.printSchema()

root
 |-- Uniq Id: string (nullable = true)
 |-- Job Title: string (nullable = false)
 |-- Job Salary: string (nullable = true)
 |-- Job Experience Required: string (nullable = true)
 |-- Key Skills: string (nullable = false)
 |-- Role Category: string (nullable = true)
 |-- Location: string (nullable = false)
 |-- Functional Area: string (nullable = false)
 |-- Industry: string (nullable = false)
 |-- Role: string (nullable = true)
 |-- Highest Salary: string (nullable = true)
 |-- Lowest Salary: string (nullable = true)
 |-- Highest Salary: string (nullable = true)
 |-- Lowest Salary: string (nullable = true)
 |-- Min Experience Required: string (nullable = true)
 |-- Max Experience Required: string (nullable = true)



In [ ]:
df.show(truncate=False)

+--------------------------------+----------------------+----------------------------+-----------------------+---------------------------------+-------------+--------+---------------+-----------------+--------------------------------+--------------+-------------+-----------------------+-----------------------+
|Uniq Id                         |Job Title             |Job Salary                  |Job Experience Required|Key Skills                       |Role Category|Location|Functional Area|Industry         |Role                            |Highest Salary|Lowest Salary|Min Experience Required|Max Experience Required|
+--------------------------------+----------------------+----------------------------+-----------------------+---------------------------------+-------------+--------+---------------+-----------------+--------------------------------+--------------+-------------+-----------------------+-----------------------+
|9be62c49a0b7ebe982a4af1edaa7bc5f| Digital Media Planner| Not Di

In [ ]:
df.select("key Skills").show()

+--------------------+
|          key Skills|
+--------------------+
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
|[ Media Planning,...|
+--------------------+
only showing top 20 rows



In [ ]:
# Write DataFrame to a CSV file
df.write.csv("/content/job_data.csv", header=True, mode="overwrite")


AnalysisException: Column `Key Skills` has a data type of array<string>, which is not supported by CSV.

In [ ]:
# Read the CSV files using Spark
df1 = spark.read.csv("/content/job_data.csv/part-00000-7f39cf96-5fe7-40fe-a6b8-eb85a9517c67-c000.csv", header=True)
df2 = spark.read.csv("/content/job_data.csv/part-00001-7f39cf96-5fe7-40fe-a6b8-eb85a9517c67-c000.csv", header=True)

# Concatenate the DataFrames
df_combined = df1.union(df2)

# Convert Spark DataFrame to pandas DataFrame
df_pandas = df_combined.toPandas()

# Print the pandas DataFrame
print(df_pandas)

Py4JError: An error occurred while calling o2633.pandasStructHandlingMode. Trace:
py4j.Py4JException: Method pandasStructHandlingMode([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:321)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:329)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:829)

